![data-x](http://oi64.tinypic.com/o858n4.jpg)


# Homework 09
Sentiment Analysis on IMDB movie reviews

Reference: https://github.com/ikhlaqsidhu/data-x/blob/master/07-tools-webscraping-crawling-nlp-sentiment-sc1t/notebook-nlp-sentiment-analysis-imdb-afo_v1.ipynb



# As you go through the notebook, you will encounter these main steps in the code: 
1. Reading of file labeledTrainData.tsv from data folder in a dataframe `train`.
2. A function review_cleaner(train['review'],lemmatize,stem) which cleans the reviews in the input file.
3. A function train_predict_sentiment(cleaned_reviews, y=train["sentiment"],ngram=1,max_features=1000
4. You will see a model has been trained on unigrams of the reviews without lemmatizing and stemming.
5. Your task is in 5.TODO section.

In [1]:
# Remove warnings
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

#make compatible with Python 2 and Python 3
from __future__ import print_function, division, absolute_import 


## Data set

The labeled training data set consists of 25,000 IMDB movie reviews. There is also an unlabeled test set with 25,000 IMDB movie reviews. The sentiment of the reviews are binary, meaning an IMDB rating < 5 results in a sentiment score of 0, and a rating >=7 have a sentiment score of 1 (no reviews with score 5 or 6 are included in the analysis). No individual movie has more than 30 reviews.

## File description

* **labeledTrainData** - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review. 

* **testData** - The unlabeled test set. 25,000 rows containing an id, and text for each review. 

## Data columns
* **id** - Unique ID of each review
* **sentiment** - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* **review** - Text of the review


## 1. Data set statistics


In [2]:
import numpy as np
import pandas as pd       
train = pd.read_csv("data/labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)
# train.shape should be (25000,3)

In [3]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
#import packages

import bs4 as bs
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
# nltk.download('all')
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re

from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

<div id='sec3'></div>
##  2.Preparing the data set for classification



We'll create a function called `review_cleaner` that reads in a review and:

- Removes HTML tags (using beautifulsoup)
- **Extract emoticons (emotion symbols, aka smileys :D )**
- Removes non-letters (using regular expression)
- Converts all words to lowercase letters and tokenizes them (using .split() method on the review strings, so that every word in the review is an element in a list)
- Removes all the English stopwords from the list of movie review words
- Join the words back into one string seperated by space, append the emoticons to the end

**NOTE: Transform the list of stopwords to a set before removing the stopwords. I.e. assign `eng_stopwords = set(stopwords.words("english"))`. Use the set to look up stopwords. This will speed up the computations A LOT (Python is much quicker when searching a set than a list).**

In [28]:
# 1. 
from nltk.corpus import stopwords
from nltk.util import ngrams


ps = PorterStemmer()
wnl = WordNetLemmatizer()


def review_cleaner(reviews,lemmatize=True,stem=False):
    '''
    Clean and preprocess a review.

    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split reviews
    4. Remove English stopwords
    5. Rejoin to one string
    '''
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()
        #1. Remove HTML tags
    
    cleaned_reviews=[]
    for i,review in enumerate(train['review']):
    # print progress
        if( (i+1)%500 == 0 ):
            print("Done with %d reviews" %(i+1))
        review = bs.BeautifulSoup(review).text

        #2. Use regex to find emoticons
        emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review)

        #3. Remove punctuation
        review = re.sub("[^a-zA-Z]", " ",review)

        #4. Tokenize into words (all lower case)
        review = review.lower().split()

        #5. Remove stopwords
        eng_stopwords = set(stopwords.words("english"))
            
        clean_review=[]
        for word in review:
            if word not in eng_stopwords:
                if lemmatize is True:
                    word=wnl.lemmatize(word)
                elif stem is True:
                    if word == 'oed':
                        continue
                    word=ps.stem(word)
                clean_review.append(word)

        #6. Join the review to one sentence
        
        review_processed = ' '.join(clean_review+emoticons)
        cleaned_reviews.append(review_processed)
    

    return(cleaned_reviews)

##  3. Function to train and validate a sentiment analysis model using Random Forest Classifier

In [29]:
from sklearn.ensemble import RandomForestClassifier
# # CountVectorizer can actucally handle a lot of the preprocessing for us
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics # for confusion matrix, accuracy score etc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix


np.random.seed(0)


def train_predict_sentiment(cleaned_reviews, y=train["sentiment"],ngram=1,max_features=1000):
    '''This function will:
    1. split data into train and test set.
    2. get n-gram counts from cleaned reviews 
    3. train a random forest model using train n-gram counts and y (labels)
    4. test the model on your test split
    5. print accuracy of sentiment prediction on test and training data
    6. print confusion matrix on test data results

    To change n-gram type, set value of ngram argument
    To change the number of features you want the countvectorizer to generate, set the value of max_features argument'''

    print("Creating the bag of words model!\n")
    # CountVectorizer" is scikit-learn's bag of words tool, here we show more keywords 
    vectorizer = CountVectorizer(ngram_range=(1, ngram),analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = max_features) 
    
    X_train, X_test, y_train, y_test = train_test_split(\
    cleaned_reviews, y, random_state=0, test_size=.2)

    # Then we use fit_transform() to fit the model / learn the vocabulary,
    # then transform the data into feature vectors.
    # The input should be a list of strings. .toarraty() converts to a numpy array
    
    train_bag = vectorizer.fit_transform(X_train).toarray()
    test_bag = vectorizer.transform(X_test).toarray()
    #print('TOP 20 FEATURES ARE: ',(vectorizer.get_feature_names()[:20]))


    print("Training the random forest classifier!\n")
    # Initialize a Random Forest classifier with 75 trees
    forest = RandomForestClassifier(n_estimators = 50) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the target variable
    forest = forest.fit(train_bag, y_train)


    train_predictions = forest.predict(train_bag)
    test_predictions = forest.predict(test_bag)
    
    train_acc = metrics.accuracy_score(y_train, train_predictions)
    valid_acc = metrics.accuracy_score(y_test, test_predictions)
    print(" The training accuracy is: ", train_acc, "\n", "The validation accuracy is: ", valid_acc)
    print()
    print('CONFUSION MATRIX:')
    print('         Predicted')
    print('          neg pos')
    print(' Actual')
    c=confusion_matrix(y_test, test_predictions)
    print('     neg  ',c[0])
    print('     pos  ',c[1])

    #Extract feature importnace
    print('\nTOP TEN IMPORTANT FEATURES:')
    importances = forest.feature_importances_
    indices = np.argsort(importances)[::-1]
    top_10 = indices[:20]
    print([vectorizer.get_feature_names()[ind] for ind in top_10])

## 4. Train and test  Model on the IMDB data

In [7]:
#Clean the reviews in the training set 'train' using review_cleaner function defined above
# Here we use the original reviews without lemmatizing and stemming

original_clean_reviews=review_cleaner(train['review'],lemmatize=False,stem=False)
train_predict_sentiment(cleaned_reviews=original_clean_reviews, y=train["sentiment"],ngram=1,max_features=1000)

Done with 500 reviews
Done with 1000 reviews
Done with 1500 reviews
Done with 2000 reviews
Done with 2500 reviews
Done with 3000 reviews
Done with 3500 reviews
Done with 4000 reviews
Done with 4500 reviews
Done with 5000 reviews
Done with 5500 reviews
Done with 6000 reviews
Done with 6500 reviews
Done with 7000 reviews
Done with 7500 reviews
Done with 8000 reviews
Done with 8500 reviews
Done with 9000 reviews
Done with 9500 reviews
Done with 10000 reviews
Done with 10500 reviews
Done with 11000 reviews
Done with 11500 reviews
Done with 12000 reviews
Done with 12500 reviews
Done with 13000 reviews
Done with 13500 reviews
Done with 14000 reviews
Done with 14500 reviews
Done with 15000 reviews
Done with 15500 reviews
Done with 16000 reviews
Done with 16500 reviews
Done with 17000 reviews
Done with 17500 reviews
Done with 18000 reviews
Done with 18500 reviews
Done with 19000 reviews
Done with 19500 reviews
Done with 20000 reviews
Done with 20500 reviews
Done with 21000 reviews
Done with 21

In [11]:
train_predict_sentiment(cleaned_reviews=original_clean_reviews, y=train["sentiment"],ngram=2,max_features=1000)

Creating the bag of words model!

Training the random forest classifier!

 The training accuracy is:  0.99995 
 The validation accuracy is:  0.8208

CONFUSION MATRIX:
         Predicted
          neg pos
 Actual
     neg   [2121  427]
     pos   [ 469 1983]

TOP TEN IMPORTANT FEATURES:
['bad', 'worst', 'great', 'awful', 'excellent', 'waste', 'terrible', 'wonderful', 'boring', 'best', 'nothing', 'worse', 'poor', 'love', 'minutes', 'horrible', 'stupid', 'waste time', 'movie', 'even']


# 5. TODO: 
To do this exercise you only need to change argument values in the functions review_cleaner and train_predict_semtiment.

1. In __UNIGRAM setting__ ie. when ngram=1 in the function `train_predict_sentiment()`.Compare the performance of original cleaned reviews in Sentiment anlysis to 
    1. lemmatized reviews 
    2. stemmed reviews
2. In __BIGRAM setting__ ie. when ngram=2 in the function `train_predict_sentiment()`.Compare the performance of original cleaned reviews in sentiment analysis to:
     1. lemmatized reviews
     2. stemmed reviews
3. In __UNIGRAM setting__  and _lemmatize=True__ ie. when ngram=1, compare the performance of Sentiment analysis for these values of maximum features=[10,100,1000,5000], you can change the value of argument max_features in `train_predict_sentiment()
    
### SUBMISSION:  For each question in 5. TODO  report your results in a PDF. Write a 100-200 word summary of your observations overall.
### Mention the  review_cleaner( ) and train_predict_sentiment( ) argument setting that you used in each case. Do not submit any ipython notebook.



Eg: For original review with unigram and 5000 max_features, I will report:

original_clean_reviews=review_cleaner(train['review'],lemmatize=False,stem=False)
train_predict_sentiment(cleaned_reviews=original_clean_reviews, y=train["sentiment"],ngram=1,max_features=5000)

The training accuracy is:  1.0 
The validation accuracy is:  0.836


**Name:** Wil Aquino

**Date:** October 28, 2021

**Project:** Feature Engineering & Text Representation

**Note:** This work is based off of Module 510 from Data X at Berkeley.

In [30]:
# import needed libraries
import numpy as np
import pandas as pd

import bs4 as bs
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
# nltk.download('all')
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re

from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

# NOTE: The only Jupyter cells ran above are the cells with 
#       review_cleaner() and train_predict_sentiment()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Wil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
# use a part of our project data
reviews = pd.read_csv("chevron_reviews.csv", header = 0, sep = ";")
reviews.head()

,date,employee_title,employee_status,review_title,pros,cons
0,2021-10-15,Operator,Current Employee,Great place to work,Because of the work culture\n,Swing shift and extended work hours\nBe the fi...
1,2021-10-11,Process Engineer,"Current Employee, more than 8 years",Great Compnay,Inclusive and diverse company! Great work and ...,I don't have nothing to share.\nBe the first t...
2,2021-10-10,Team Member,"Former Employee, less than 1 year",Excellent,"Great job, excellent crew and perks. I miss it.\n",It was the third best job I ever had.\nVerify ...
3,2021-10-06,Reliability Engineer,Current Employee,Great Place to Work,Excellent work life balance Very diverse and i...,Promotions can be slow moving\nVerify your ema...
4,2021-10-06,Sales Associate/Cashier,Current Employee,It’s cool Ig,Good pay Time off Good customers I don’t know ...,Bad managers Horrible coworkers Dirty all the ...


In [32]:
prelim_clean_reviews = reviews.drop(["date", "employee_title", "employee_status"], axis = 1)
prelim_clean_reviews.head()

,review_title,pros,cons
0,Great place to work,Because of the work culture\n,Swing shift and extended work hours\nBe the fi...
1,Great Compnay,Inclusive and diverse company! Great work and ...,I don't have nothing to share.\nBe the first t...
2,Excellent,"Great job, excellent crew and perks. I miss it.\n",It was the third best job I ever had.\nVerify ...
3,Great Place to Work,Excellent work life balance Very diverse and i...,Promotions can be slow moving\nVerify your ema...
4,It’s cool Ig,Good pay Time off Good customers I don’t know ...,Bad managers Horrible coworkers Dirty all the ...


In [33]:
# remove newline characters
prelim_clean_reviews['pros'] = prelim_clean_reviews['pros'].str.replace('\n', '', regex = True)
prelim_clean_reviews['cons'] = prelim_clean_reviews['cons'].str.replace('\n', '', regex = True)

# remove bad strings caught by the web scraper
badString1 = "Verify your email to continue reading or Resend email"
badString2 = "Be the first to find this review helpfulHelpfulShareRepor"

prelim_clean_reviews['cons'] = prelim_clean_reviews['cons'].str.replace(badString1, '', regex = True)
prelim_clean_reviews['cons'] = prelim_clean_reviews['cons'].str.replace(badString2, '', regex = True)

# compile pros and cons together into a column
prelim_clean_reviews["review"] = "{ Pros. " + prelim_clean_reviews['pros'] + " } { Cons. " + prelim_clean_reviews['cons'] + " }"
clean_reviews = prelim_clean_reviews.drop(["pros", "cons"], axis = 1)

# remove empty reviews
clean_reviews = clean_reviews.dropna(subset = ['review'])
clean_reviews = clean_reviews.reset_index(drop = True)

clean_reviews

,review_title,review
0,Great place to work,{ Pros. Because of the work culture } { Cons. ...
1,Great Compnay,{ Pros. Inclusive and diverse company! Great w...
2,Excellent,"{ Pros. Great job, excellent crew and perks. I..."
3,Great Place to Work,{ Pros. Excellent work life balance Very diver...
4,It’s cool Ig,{ Pros. Good pay Time off Good customers I don...
...,...,...
984,Mixed Bag,{ Pros. - Pay and benefits - Safety culture (i...
985,Customer Service Representative,"{ Pros. Fast pace job, work is not hard,deal w..."
986,Senior Advisor,{ Pros. Work-life is well balanced; Relatively...
987,Administrative Assistant,{ Pros. Great health benefits and good compens...


In [34]:
# retrieve basic sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def round_two_digit_string(x):
    return str(round(x, 2))

def retrieve_sentiment_analysis(df, column):
    """ Performs a sentiment analysis on the given data.
    
    Args:
        df::[pd.DataFrame]
            The table of given reviews and their statistics.
            
    Return:
        sentiment_df::[pd.DataFrame]
            The reviews with their sentiment analysis data.
    """
    
    analyzer = SentimentIntensityAnalyzer()
    data = {
        "text": [],
        "negative": [],
        "neutral": [],
        "positive": [],
        "compound": []
    }

    for sentence in df[column]:
        vs = analyzer.polarity_scores(sentence)

        data["text"].append(sentence)
        data["negative"].append(round_two_digit_string(vs["neg"]))
        data["neutral"].append(round_two_digit_string(vs["neu"]))
        data["positive"].append(round_two_digit_string(vs["pos"]))
        data["compound"].append(round_two_digit_string(vs["compound"]))
        
    sentiment_df = pd.DataFrame(data)
    
    return sentiment_df

sentiment_analysis = retrieve_sentiment_analysis(clean_reviews, "review")
sentiment_analysis.head()

,text,negative,neutral,positive,compound
0,{ Pros. Because of the work culture } { Cons. ...,0.0,1.0,0.0,0.0
1,{ Pros. Inclusive and diverse company! Great w...,0.07,0.75,0.17,0.54
2,"{ Pros. Great job, excellent crew and perks. I...",0.05,0.59,0.36,0.91
3,{ Pros. Excellent work life balance Very diver...,0.0,0.73,0.27,0.86
4,{ Pros. Good pay Time off Good customers I don...,0.31,0.53,0.16,-0.67


In [35]:
clean_reviews["sentiment"] = sentiment_analysis["compound"]
clean_reviews.head()

,review_title,review,sentiment
0,Great place to work,{ Pros. Because of the work culture } { Cons. ...,0.0
1,Great Compnay,{ Pros. Inclusive and diverse company! Great w...,0.54
2,Excellent,"{ Pros. Great job, excellent crew and perks. I...",0.91
3,Great Place to Work,{ Pros. Excellent work life balance Very diver...,0.86
4,It’s cool Ig,{ Pros. Good pay Time off Good customers I don...,-0.67


In [36]:
# original test
train = clean_reviews
original_clean_reviews = review_cleaner(train["review"], lemmatize = False, stem = False)
train_predict_sentiment(cleaned_reviews = original_clean_reviews, y = train["sentiment"], ngram = 1, max_features = 100)

Done with 500 reviews
Creating the bag of words model!

Training the random forest classifier!

 The training accuracy is:  0.9987357774968394 
 The validation accuracy is:  0.09595959595959595

CONFUSION MATRIX:
         Predicted
          neg pos
 Actual
     neg   [0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
     pos   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

TOP TEN IMPORTANT FEATURES:
['work', 'great', 'company', 'good', 'people', 'culture', 'pay', 'management', 'benefits', 'opportunities', 'job', 'employees', 'time', 'balance', 'life', 'get', 'hours', 'slow', 'salary', 'environment']


In [37]:
# Q1 - Unigram (ngram = 1)
print("Q1 Unigram Lemmatized Reviews\n============")
new_clean_reviews = review_cleaner(train["review"], lemmatize = True, stem = False)
train_predict_sentiment(cleaned_reviews = new_clean_reviews, y = train["sentiment"], ngram = 1, max_features = 100)
print("----\n")

print("Q1 Unigram Stemmed Reviews\n============")
new_clean_reviews = review_cleaner(train["review"], lemmatize = False, stem = True)
train_predict_sentiment(cleaned_reviews = new_clean_reviews, y = train["sentiment"], ngram = 1, max_features = 100)
print("----\n")

# Q2 - Bigram (ngram = 2)
print("Q2 Bigram Lemmatized Reviews\n============")
new_clean_reviews = review_cleaner(train["review"], lemmatize = True, stem = False)
train_predict_sentiment(cleaned_reviews = new_clean_reviews, y = train["sentiment"], ngram = 2, max_features = 100)
print("----\n")

print("Q2 Bigram Stemmed Reviews\n============")
new_clean_reviews = review_cleaner(train["review"], lemmatize = False, stem = True)
train_predict_sentiment(cleaned_reviews = new_clean_reviews, y = train["sentiment"], ngram = 2, max_features = 100)
print("----\n")

# Q3 - Multiple Unigrams
for i in [10, 100, 1000, 5000]:
    print(f"Q3 Unigram Reviews with {i} Features\n============")
    new_clean_reviews = review_cleaner(train["review"], lemmatize = True, stem = False)
    train_predict_sentiment(cleaned_reviews = new_clean_reviews, y = train["sentiment"], ngram = 1, max_features = i)
    print("----\n")

Q1 Unigram Lemmatized Reviews
Done with 500 reviews
Creating the bag of words model!

Training the random forest classifier!

 The training accuracy is:  0.9987357774968394 
 The validation accuracy is:  0.050505050505050504

CONFUSION MATRIX:
         Predicted
          neg pos
 Actual
     neg   [0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
     pos   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

TOP TEN IMPORTANT FEATURES:
['work', 'good', 'company', 'people', 'great', 'pay', 'culture', 'employee', 'management', 'job', 'benefit', 'time', 'opportunity', 'lot', 'life', 'salary', 'balance', 'hour', 'environment', 'oil']
----

Q1 Unigram Stemmed Reviews
Done with 500 reviews
Creating th